In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%cd ..

/Users/xiaoyangsong/Desktop/Offline-RL-Controller-in-AM


In [4]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import os

In [6]:
def extract_single_trajectory(trajectory_id, trajectory_length=8):
    trajectory = []
    for j in range(trajectory_length):
        filename = os.path.join('..', 'RL Controller', 'RL_Dataset', f'trajectory_00{trajectory_id}', f'layer_{j+1}_data.mat')
        data = loadmat(filename)
        # Extract s, a, r
        ss = data['SS_action'][0][0]
        lp = data['LP_action'][0][0]
        u = np.array(data['uFinal'])
        uAll = data['uAll']
        r = -data['meanDeviation'][0][0]
        # Append
        sar = [u, lp, r]
        print(u.shape, lp.shape, ss.shape, r.shape, lp, ss, r)
        trajectory.append(sar)
    return trajectory

trajectory = extract_single_trajectory(trajectory_id=1, trajectory_length=8)

(1053, 1) () () () 435.21418112695454 730.7743435678826 -0.36122621301829466
(1053, 1) () () () 126.19661470414442 1421.1915508818497 -0.37706539554349666
(1053, 1) () () () 489.385336080055 979.1369092305144 -3.486651536021735
(1053, 1) () () () 504.76113742873747 1475.521122545086 -7.3816908121369105
(1053, 1) () () () 152.88703918942826 1252.1645764000257 -3.3861466576663957
(1053, 1) () () () 513.6671540129037 879.1502090256 -8.104272687881398
(1053, 1) () () () 134.9943433746089 415.3986297303739 -3.9907598469614585
(1053, 1) () () () 517.8947077982968 929.7149581288493 -9.836594728975825


In [8]:
def gather_dataset(id_list, trajectory_length=8):
    dataset = []
    for trajectory_id in id_list:
        traj = extract_single_trajectory(trajectory_id, trajectory_length)
        dataset.append(traj)
    return dataset

id_list = [1, 2, 3]
dataset = gather_dataset(id_list, trajectory_length=8)

(1053, 1) () () () 435.21418112695454 730.7743435678826 -0.36122621301829466
(1053, 1) () () () 126.19661470414442 1421.1915508818497 -0.37706539554349666
(1053, 1) () () () 489.385336080055 979.1369092305144 -3.486651536021735
(1053, 1) () () () 504.76113742873747 1475.521122545086 -7.3816908121369105
(1053, 1) () () () 152.88703918942826 1252.1645764000257 -3.3861466576663957
(1053, 1) () () () 513.6671540129037 879.1502090256 -8.104272687881398
(1053, 1) () () () 134.9943433746089 415.3986297303739 -3.9907598469614585
(1053, 1) () () () 517.8947077982968 929.7149581288493 -9.836594728975825
(1053, 1) () () () 301.77164668602336 581.0718911787624 -0.5544212187870767
(1053, 1) () () () 181.71106387504352 1352.0131111735564 -0.36400716823206797
(1053, 1) () () () 175.8699196795555 1300.2377529680389 -0.18720062542809024
(1053, 1) () () () 312.40573743068705 904.7208555560082 -1.9851228930730056
(1053, 1) () () () 498.11162169650294 580.5579760932429 -6.678266150879865
(1053, 1) () () (

In [9]:
len(dataset)

3

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# ============================
# Q-network
# ============================
class QNet(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + 1, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        return self.net(x)


# ============================
# Policy network: a = π(s)
# ============================
class PolicyNet(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )

    def forward(self, s):
        return self.net(s)


# =============================================
# Dataset loader from list-of-trajectories
# =============================================
def flatten_dataset(trajectories):
    states, actions, rewards, next_states = [], [], [], []

    for traj in trajectories:
        for t in range(len(traj) - 1):
            s, a, r = traj[t]
            s2, _, _ = traj[t+1]

            states.append(s)
            actions.append(a)
            rewards.append(r)
            next_states.append(s2)

    return (
        torch.tensor(np.array(states), dtype=torch.float32).to(device),
        torch.tensor(np.array(actions), dtype=torch.float32).unsqueeze(-1).to(device),
        torch.tensor(np.array(rewards), dtype=torch.float32).unsqueeze(-1).to(device),
        torch.tensor(np.array(next_states), dtype=torch.float32).to(device),
    )


# =============================================
# Offline Q-Learning loop (Fitted Q Iteration)
# =============================================
def train_fqi(trajectories, state_dim, gamma=0.99, K=50):
    # Load flattened dataset
    S, A, R, S2 = flatten_dataset(trajectories)
    S = S.squeeze()
    S2 = S2.squeeze()

    qnet = QNet(state_dim).to(device)
    policy = PolicyNet(state_dim).to(device)

    q_opt = optim.Adam(qnet.parameters(), lr=1e-3)
    p_opt = optim.Adam(policy.parameters(), lr=1e-4)

    for k in range(K):
        # ----------------------
        # Policy Improvement: update π so that it maximizes Q
        # ----------------------
        p_opt.zero_grad()
        a_hat = policy(S)                  # predicted continuous actions
        q_val = qnet(S, a_hat)
        loss_policy = -q_val.mean()        # maximize Q => minimize -Q
        loss_policy.backward()
        p_opt.step()

        # ----------------------
        # Q-iteration step
        # ----------------------
        with torch.no_grad():
            next_action = policy(S2)
            target = R + gamma * qnet(S2, next_action)

        q_opt.zero_grad()
        q_pred = qnet(S, A)
        loss_q = nn.MSELoss()(q_pred, target)
        loss_q.backward()
        q_opt.step()

        print(f"Iter {k}:  Q Loss={loss_q.item():.4f},  Policy Loss={loss_policy.item():.4f}")

    return qnet, policy


In [25]:
state_dim = 1053
qnet, policy = train_fqi(dataset, state_dim)


Iter 0:  Q Loss=1095.5637,  Policy Loss=111.6001
Iter 1:  Q Loss=662120.6875,  Policy Loss=3234.4185
Iter 2:  Q Loss=2838951.2500,  Policy Loss=6800.5698
Iter 3:  Q Loss=8134104.5000,  Policy Loss=11667.6484
Iter 4:  Q Loss=18853762.0000,  Policy Loss=17920.1465
Iter 5:  Q Loss=38560360.0000,  Policy Loss=25776.9375
Iter 6:  Q Loss=72388072.0000,  Policy Loss=35478.0039
Iter 7:  Q Loss=127500544.0000,  Policy Loss=47260.9844
Iter 8:  Q Loss=213547568.0000,  Policy Loss=61347.5430
Iter 9:  Q Loss=343456928.0000,  Policy Loss=77997.5625
Iter 10:  Q Loss=534244704.0000,  Policy Loss=97491.4219
Iter 11:  Q Loss=808068544.0000,  Policy Loss=120132.7656
Iter 12:  Q Loss=1193498752.0000,  Policy Loss=146250.5781
Iter 13:  Q Loss=1727051520.0000,  Policy Loss=176200.0781
Iter 14:  Q Loss=2454903040.0000,  Policy Loss=210363.4219
Iter 15:  Q Loss=3434707456.0000,  Policy Loss=249149.7656
Iter 16:  Q Loss=4738453504.0000,  Policy Loss=292995.2812
Iter 17:  Q Loss=6455211520.0000,  Policy Loss=34